In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel


X_train = np.load('initial_data/function_6/initial_inputs.npy')
Y_train = np.load('initial_data/function_6/initial_outputs.npy')
#[1.00000e-06 6.66666e-01 2.22222e-01 7.77777e-01 3.33333e-01]

X_train = np.append(X_train, [[0.999999,0.666666,0.222222,0.777777,0.333333],
                              [0.999999,0.111111,0.555555,0.888888,0.111111],
                              [9.99999e-01,2.22222e-01,7.77777e-01,6.66666e-01,1.00000e-06],
                              [0.222222,0.333333,0.666666,0.777777,0.111111]
], axis=0)
Y_train = np.append(Y_train, [-1.33525312859238,-1.04667781187405,-1.01192954015728,0.182491836305764])

print(X_train)
print(Y_train)

[[7.28186105e-01 1.54692570e-01 7.32551669e-01 6.93996509e-01
  5.64013105e-02]
 [2.42384347e-01 8.44099972e-01 5.77809099e-01 6.79021284e-01
  5.01952888e-01]
 [7.29522610e-01 7.48106200e-01 6.79774641e-01 3.56552279e-01
  6.71053683e-01]
 [7.70620242e-01 1.14403744e-01 4.67799319e-02 6.48324285e-01
  2.73549053e-01]
 [6.18812299e-01 3.31802137e-01 1.87287868e-01 7.56238474e-01
  3.28834798e-01]
 [7.84958094e-01 9.10682349e-01 7.08120104e-01 9.59225429e-01
  4.91149586e-03]
 [1.45110786e-01 8.96684598e-01 8.96322235e-01 7.26271537e-01
  2.36271991e-01]
 [9.45069068e-01 2.88459051e-01 9.78805764e-01 9.61655587e-01
  5.98015936e-01]
 [1.25720155e-01 8.62724692e-01 2.85443322e-02 2.46605272e-01
  7.51206241e-01]
 [7.57594355e-01 3.55831415e-01 1.65228997e-02 4.34207205e-01
  1.12433044e-01]
 [5.36796903e-01 3.08780907e-01 4.11879285e-01 3.88225177e-01
  5.22528304e-01]
 [9.57739669e-01 2.35668572e-01 9.91458496e-02 1.56805934e-01
  7.13173731e-02]
 [6.29307895e-01 8.03483678e-01 8.114084

In [2]:
kernel = RBF(length_scale=np.ones(5)) + WhiteKernel(noise_level=1)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-5, normalize_y=True)
gp.fit(X_train, Y_train.ravel())

C:\Code\RoughWork\Jupyter\venv\lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(alpha=1e-05,
                         kernel=RBF(length_scale=[1, 1, 1, 1, 1]) + WhiteKernel(noise_level=1),
                         normalize_y=True)

In [3]:
def ucb(X, gp, kappa=1.96):  # kappa can be adjusted based on your confidence level needs
    mean, std = gp.predict(X, return_std=True)
    return mean + kappa * std

num_points = 10  # Adjust based on computational resources
test_range = np.linspace(0, 1, num_points)
xx, yy, zz, ww, vv = np.meshgrid(test_range, test_range, test_range, test_range, test_range)
all_combinations = np.vstack([xx.ravel(), yy.ravel(), zz.ravel(), ww.ravel(), vv.ravel()]).T

# Calculate UCB for all combinations
UCB_values = ucb(all_combinations, gp)

In [4]:
indices_of_max_ucb = np.argsort(-UCB_values)[:100]  # Top 100 for robust range estimate
optimal_points = all_combinations[indices_of_max_ucb]

# Calculate potential ranges
ranges = [np.percentile(optimal_points[:, i], [5, 95]) for i in range(5)]

In [5]:
print("Proposed ranges for each ingredient based on UCB:")
for i, range_values in enumerate(ranges, start=1):
    print(f"Ingredient {i}: {range_values}")

Proposed ranges for each ingredient based on UCB:
Ingredient 1: [0.11111111 0.22222222]
Ingredient 2: [0.11111111 0.33333333]
Ingredient 3: [0.11111111 0.77777778]
Ingredient 4: [0.77777778 1.        ]
Ingredient 5: [0.         0.11111111]
